In [55]:
import csv
import numpy as np
import pandas as pd
import string

%matplotlib inline

# Functions

In [58]:
def format_table_output(raw, title = None):
    pre_text = '\\begin{table}\n\\small\n' 
    
    if title:
        pre_text = pre_text + '\\caption*{' + title + '}\n'
    
    post_text = '\\end{table}'
    return pre_text + raw + post_text

# Load Data

In [90]:
# Processed disclosure data
data_605 = pd.read_csv('../data/processed/605_processed.csv')
data_606 = pd.read_csv('../data/processed/606_processed.csv')

rawdata_605 = pd.read_csv('../data/rawdata_605.csv')
rawdata_606 = pd.read_csv('../data/rawdata_606.csv')

# Panel data
data_panel = pd.read_csv('../data/processed/regression_data_levels.csv')
data_panel.head()

,MarketCenter,Quarter,Exchange,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,...,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT,Broker,Rebate,MktShare,Rebate_Dummy,Broker_Size
0,BNYC,2015Q1,NASDAQ,11,213581,43704397,70249,43628268,0,43526470,...,0.115073,0.011198,-0.005493,0.105550,0.134962,Insigneo Securities,0.0,0.7766,0,NaN
1,BNYC,2015Q1,NASDAQ,12,37042,7770176,4851,7750829,0,7713717,...,0.141117,0.005108,-0.004517,1.566054,0.528654,Insigneo Securities,0.0,0.6279,0,NaN
2,BNYC,2015Q1,NYSE,11,396811,81015802,108105,80902077,0,80833088,...,0.070049,0.006949,-0.002302,0.094201,0.074817,Insigneo Securities,0.0,0.7379,0,NaN
3,BNYC,2015Q1,NYSE,12,45238,9529134,5233,9506679,0,9478187,...,0.016105,0.003145,-0.002832,1.285204,-0.294374,Insigneo Securities,0.0,0.5228,0,NaN
4,BNYC,2015Q1,Other,11,257920,55831472,108233,55715793,0,55640416,...,-0.023154,0.008315,-0.003278,0.331655,-0.098908,Insigneo Securities,0.0,0.7026,0,NaN


# Summary Stats

## Market Centers

In [91]:
data_605_dsc = pd.DataFrame()
data_605['MarketCenter'].unique()
data_605 = data_605.query('Quarter == "2017Q4"')
data_605.head()

,MarketCenter,Quarter,Exchange,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,...,PrImp_ExpAmt,All_AvgT,MktCtrAvg_PrImp_Pct,Rel_PrImp_Pct,MktCtrAvg_PrImp_AvgT,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT
165,BNYC,2017Q4,NASDAQ,11,161006,32758843,62613,32693477,0,32673175,...,0.012507,0.191631,0.900613,-0.032599,0.029092,0.113608,0.015328,-0.002821,0.039196,0.152435
166,BNYC,2017Q4,NASDAQ,12,34626,7432429,153175,7277851,0,7246145,...,0.002645,1.444451,0.594646,-0.078809,0.097610,0.066265,0.005737,-0.003092,1.130199,0.314252
167,BNYC,2017Q4,NASDAQ,13,5661,1112726,225083,858180,0,528610,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000
168,BNYC,2017Q4,NASDAQ,14,5597,1179371,325757,806803,0,203497,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000
169,BNYC,2017Q4,NASDAQ,15,13207,2744983,954314,1445084,0,142530,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000


### Dates

In [92]:
data_605_dsc['First_Quarter_obs'] = data_605.groupby(['MarketCenter']).first()['Quarter']
data_605_dsc['Last_Quarter_obs']  = data_605.groupby(['MarketCenter']).last()['Quarter']
data_605_dsc = data_605_dsc.reset_index()
data_605_dsc

,MarketCenter,First_Quarter_obs,Last_Quarter_obs
0,BNYC,2017Q4,2017Q4
1,CDRG,2017Q4,2017Q4
2,FBCO,2017Q4,2017Q4
3,G1ES,2017Q4,2017Q4
4,SGMA,2017Q4,2017Q4
5,UBSS,2017Q4,2017Q4
6,VRTU,2017Q4,2017Q4
7,WOLV,2017Q4,2017Q4


### Descriptive

In [93]:
descriptive_vars = ['MktCtrExecShares', 'AwayExecShares', 'PrImp_AvgAmt', 'PrImp_Pct', 'PrImp_ExpAmt', 'PrImp_AvgT', 'All_AvgT', 'AvgEffecSpread']

temp = data_605.groupby(['MarketCenter', 'Exchange', 'OrderType']).mean()[descriptive_vars].reset_index()
data_605_dsc = data_605_dsc.merge(temp)
data_605_dsc.head()

,MarketCenter,First_Quarter_obs,Last_Quarter_obs,Exchange,OrderType,MktCtrExecShares,AwayExecShares,PrImp_AvgAmt,PrImp_Pct,PrImp_ExpAmt,PrImp_AvgT,All_AvgT,AvgEffecSpread
0,BNYC,2017Q4,2017Q4,NASDAQ,Limit,7277851.0,0.0,0.005127,0.515837,0.002645,0.163875,1.444451,0.036997
1,BNYC,2017Q4,2017Q4,NASDAQ,Market,32693477.0,0.0,0.014408,0.868015,0.012507,0.142701,0.191631,0.020463
2,BNYC,2017Q4,2017Q4,NASDAQ,Other,1036689.0,0.0,NaN,0.000000,NaN,NaN,0.000000,0.000000
3,BNYC,2017Q4,2017Q4,NYSE,Limit,7616934.0,0.0,0.003185,0.602457,0.001919,0.154974,1.120218,0.020033
4,BNYC,2017Q4,2017Q4,NYSE,Market,61096944.0,0.0,0.007877,0.908111,0.007153,0.139359,0.169840,0.009888


In [94]:
panels = ['NASDAQ', 'NYSE', 'Other']
data_output = data_605_dsc.copy()

data_output['MktCtrExecShares'] = data_output['MktCtrExecShares'].apply(lambda x: np.int(x))
data_output['AwayExecShares'] = data_output['AwayExecShares'].apply(lambda x: np.int(x))
data_output['PrImp_Pct'] = data_output['PrImp_Pct'].apply(lambda x: str(np.round(x*100, decimals = 2)) + '%')

for i in range(0, len(panels)):
   
    data_table = data_output.query('Exchange == "' + panels[i] + '"').query('OrderType == "Market"')
    data_table = data_table[['MarketCenter'] + descriptive_vars]
    raw_latex_table = data_table.to_latex(index = False)
    
    table_header = 'Panel ' + string.ascii_uppercase[i] + ': ' + panels[i] + ' Stocks'
    
    print(format_table_output(raw_latex_table, title = table_header), end = '\n\n')

\begin{table}
\small
\caption*{Panel A: NASDAQ Stocks}
\begin{tabular}{lrrrlrrrr}
\toprule
MarketCenter &  MktCtrExecShares &  AwayExecShares &  PrImp\_AvgAmt & PrImp\_Pct &  PrImp\_ExpAmt &  PrImp\_AvgT &  All\_AvgT &  AvgEffecSpread \\
\midrule
        BNYC &          32693477 &               0 &      0.014408 &     86.8\% &      0.012507 &    0.142701 &  0.191631 &        0.020463 \\
        CDRG &         210788147 &          251506 &      0.024368 &    93.16\% &      0.022702 &    0.005730 &  0.005808 &        0.012677 \\
        FBCO &             70391 &         3874649 &      0.006053 &    84.15\% &      0.005093 &    0.004580 &  0.005972 &        0.059400 \\
        G1ES &          91879401 &               0 &      0.024455 &     94.9\% &      0.023208 &    0.007133 &  0.011400 &        0.015366 \\
        SGMA &          72691153 &               0 &      0.015809 &    87.26\% &      0.013795 &    0.001120 &  0.001922 &        0.022109 \\
        UBSS &          22122312 &    

## Brokers

In [110]:
# Find out which brokers accept rebates
broker_rebate_dict = {}

for broker in data_606['Broker'].unique():
    
    broker_rebate_dict[broker] = data_606[data_606['Broker'] == broker]['Rebate_Dummy'].iloc[0]
    
#brokers_paid = [brk for brk, rb in]

brokers_paid = [brk for brk in broker_rebate_dict if broker_rebate_dict[brk] == 1]
brokers_unpaid = [brk for brk in broker_rebate_dict if broker_rebate_dict[brk] == 0]

In [111]:
data_brokers_dsc = pd.DataFrame()

In [112]:
brokers = data_panel['Broker'].unique()

for broker in brokers:
    
    broker_rebate_dummy = data_panel.query('Broker == "' + broker + '"').iloc[0]['Rebate_Dummy']
    broker_connected_mktctrs = len(list(data_panel.query('Broker == "' + broker + '"')['MarketCenter'].unique()))
    
    data_brokers_dsc = data_brokers_dsc.append(
        {'Broker': broker, 'Rebate_Dummy': broker_rebate_dummy, 'MktCtr_Universe_Size': broker_connected_mktctrs}, 
        ignore_index=True)
    
    
data_brokers_dsc

,Broker,MktCtr_Universe_Size,Rebate_Dummy
0,Insigneo Securities,6.0,0.0
1,Bank of the West,6.0,0.0
2,Deutsche,6.0,1.0
3,Boenning Scattergood,7.0,1.0
4,Edward Jones,7.0,0.0
5,Credit Suisse,6.0,1.0
6,Hollencrest Securities,6.0,1.0
7,AXA,6.0,0.0
8,BTIG,4.0,1.0
9,Euro Pacific Capital,5.0,0.0


In [113]:
mktctrs = list(data_605['MarketCenter'].unique())

data_606_mktshr = data_panel.groupby(['MarketCenter', 'Broker']).mean().reset_index()

display(data_606_mktshr.query('Broker == "AXA"'))

temp = (1-data_606_mktshr.groupby(['Broker']).sum()).reset_index()
temp['MarketCenter'] = 'zzOther'
data_606_mktshr = data_606_mktshr.append(temp)

,MarketCenter,Broker,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,ExecShares_10_29,...,MktCtrAvg_PrImp_AvgT,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT,Rebate,MktShare,Rebate_Dummy,Broker_Size
7,CDRG,AXA,11.5,7.691776e+06,1.196953e+09,6.790322e+08,5.177851e+08,6.871593e+04,5.176825e+08,85954.986111,...,0.247637,-0.123019,0.007392,0.000413,0.829694,-0.675203,0.0,0.182919,0.0,9999999.0
33,FBCO,AXA,11.5,6.789583e+04,1.383350e+07,4.810304e+04,1.322172e+07,5.630328e+05,1.377175e+07,6016.291667,...,0.201697,0.082578,0.006730,-0.002456,0.767821,0.240855,0.0,0.075254,0.0,9999999.0
48,G1ES,AXA,11.5,5.256432e+05,1.071749e+08,1.219182e+05,1.069851e+08,0.000000e+00,1.068746e+08,64811.197674,...,0.234701,-0.146926,0.007290,0.002928,0.835255,-0.348106,0.0,0.195986,0.0,9999999.0
59,SGMA,AXA,11.5,1.336170e+06,2.330731e+08,1.032962e+08,1.297738e+08,0.000000e+00,1.297377e+08,12136.117647,...,0.128498,-0.079786,0.007849,0.002856,0.730567,-0.562149,0.0,0.068274,0.0,9999999.0
75,UBSS,AXA,11.5,8.366569e+06,1.230690e+09,9.769899e+08,1.756340e+08,6.080746e+07,2.342838e+08,937121.031250,...,0.191693,0.083672,0.007342,0.000242,0.756657,0.974902,0.0,0.092428,0.0,9999999.0
101,VRTU,AXA,11.5,3.936978e+06,6.554066e+08,3.197718e+08,3.355861e+08,0.000000e+00,3.351193e+08,352245.500000,...,0.089636,0.013573,0.007866,0.000663,0.607954,-0.321408,0.0,0.115508,0.0,9999999.0


In [126]:
data_output = data_606_mktshr.copy()

temp = data_output[data_output['Broker'].apply(lambda x: x in brokers_paid)].groupby('MarketCenter').mean().reset_index().copy()
temp['Broker'] = 'zzPOF Average'
data_output = data_output.append(temp)

temp = data_output[data_output['Broker'].apply(lambda x: x in brokers_unpaid)].groupby('MarketCenter').mean().reset_index().copy()
temp['Broker'] = 'zzNon-POF Average'
data_output = data_output.append(temp)

temp = data_output.groupby('MarketCenter').mean().reset_index().copy()
temp['Broker'] = 'zzAll Average'
data_output = data_output.append(temp)

data_output['MktShare'] = data_output['MktShare'].fillna(0).apply(lambda x: np.round(x*100, decimals = 0))
data_output['MktShare'] = data_output['MktShare'].fillna(0).apply(lambda x: str(int(x)) + '%' if x > 0.00001 else '<1%')
data_output = data_output.pivot(columns = 'MarketCenter', values = 'MktShare', index = 'Broker')

#display(data_output)
#print(data_output.fillna('').to_latex().replace('<', '\\textless'))
print(data_output.loc[brokers_paid + ['zzPOF Average']].fillna('').to_latex().replace('<', '\\textless'))
print(data_output.loc[brokers_unpaid + ['zzNon-POF Average']].fillna('').to_latex().replace('<', '\\textless'))
print(data_output.loc[['zzAll Average']].fillna('').to_latex().replace('<', '\\textless'))

\begin{tabular}{llllllllll}
\toprule
MarketCenter & BNYC & CDRG & FBCO & G1ES & SGMA & UBSS & VRTU & WOLV & zzOther \\
Broker                 &      &      &      &      &      &      &      &      &         \\
\midrule
Deutsche               &   5\% &   1\% &   1\% &      &   1\% &   1\% &  \textless1\% &      &     91\% \\
Boenning Scattergood   &   2\% &  22\% &   4\% &   5\% &  10\% &  13\% &   8\% &      &     36\% \\
Evercore Group         &      &      &  11\% &      &      &      &      &      &     89\% \\
Credit Suisse          &  \textless1\% &  \textless1\% &  46\% &      &  \textless1\% &  \textless1\% &  \textless1\% &      &     53\% \\
Barclays Capital       &      &  \textless1\% &  \textless1\% &      &  \textless1\% &  \textless1\% &      &      &    100\% \\
Cambria Capital        &      &  10\% &      &      &      &  81\% &      &      &      9\% \\
JP Morgan              &      &  12\% &   5\% &      &      &  16\% &   4\% &      &     64\% \\
Inlet Securities   

In [115]:
data_output

,ATQShares,ATQ_AvgT,ATQ_Pct,ATQ_TotalT,All_AvgT,AvgEffecSpread,AvgEffecSpread_T,AvgRealSpread,AvgRealSpread_T,AwayExecShares,...,PrImp_TotalAmt,PrImp_TotalT,Rebate,Rebate_Dummy,Rel_All_AvgT,Rel_PrImp_AvgT,Rel_PrImp_ExpAmt,Rel_PrImp_Pct,index,0
0,3.136106e+06,2.628079,0.184474,5.266527e+06,0.972722,0.019846,4.329002e+05,0.013577,3.274566e+05,13.333333,...,1.708388e+05,4.641864e+06,0.0,0.0,0.113115,-0.036001,-0.003625,0.024937,NaN,NaN
1,3.288533e+06,2.758675,0.210429,6.161921e+06,1.123194,0.018267,3.858452e+05,0.011180,2.777187e+05,20.000000,...,1.700924e+05,4.648872e+06,1.0,1.0,0.379406,0.083933,-0.003219,-0.002837,NaN,NaN
2,4.818538e+06,1.536364,0.207642,6.315029e+06,0.649855,0.014282,4.592488e+05,0.009210,3.298498e+05,0.000000,...,2.087060e+05,4.806997e+06,1.0,1.0,-0.104575,0.020714,-0.002293,0.022258,NaN,NaN
3,3.076620e+06,2.624104,0.189807,5.078733e+06,0.987452,0.018401,3.727913e+05,0.012589,2.873435e+05,0.000000,...,1.504262e+05,4.046760e+06,1.0,1.0,0.035600,-0.007231,-0.003356,0.015394,NaN,NaN
4,3.381749e+06,2.359925,0.196407,5.536784e+06,0.910298,0.018844,3.915207e+05,0.012384,2.902558e+05,0.000000,...,1.631693e+05,3.440941e+06,0.0,0.0,0.092271,0.030690,-0.003483,0.017532,NaN,NaN
5,4.090221e+06,1.889291,0.224933,6.771649e+06,0.827550,0.014851,4.159452e+05,0.006480,2.731569e+05,0.000000,...,2.183252e+05,4.719967e+06,1.0,1.0,0.339949,0.067537,-0.001944,-0.006541,NaN,NaN
6,2.868075e+06,2.835344,0.168096,5.026759e+06,0.937375,0.020265,4.531174e+05,0.014425,3.491480e+05,11.111111,...,1.656635e+05,4.826665e+06,0.0,0.0,0.091662,-0.039043,-0.003588,0.043678,NaN,NaN
7,2.185263e+08,0.159545,0.360615,3.855400e+07,0.154491,0.018369,9.837135e+06,0.007103,3.819500e+06,68715.930556,...,3.446838e+06,1.686506e+07,0.0,0.0,-0.675203,-0.123019,0.000413,-0.131540,NaN,NaN
8,2.282197e+08,0.165348,0.354829,4.063401e+07,0.161816,0.017951,1.026772e+07,0.006856,3.976973e+06,74962.833333,...,3.678871e+06,1.830318e+07,0.0,0.0,-0.691012,-0.124447,0.000541,-0.130092,NaN,NaN
9,2.416423e+08,0.124896,0.386873,3.606610e+07,0.172667,0.020171,1.078349e+07,0.008910,4.582927e+06,5094.272727,...,2.902873e+06,2.567683e+07,1.0,1.0,-0.717617,-0.126022,-0.000223,-0.150995,NaN,NaN


In [54]:
data_606_brk_info = data_606.groupby(['Broker']).mean().reset_index()#['Broker', 'Rebate_Dummy'].copy()
# data_606_brk_info['First_Quarter_obs'] = data_606.groupby(['Broker']).first()['Quarter']
# data_606_brk_info['Last_Quarter_obs']  = data_606.groupby(['Broker']).last()['Quarter']
# data_606_brk_info = data_606_brk_info.reset_index().drop('index', axis = 1)
data_606_brk_info

,Broker,Rebate,MktShare,Rebate_Dummy
0,AXA,0.000000,0.156236,0
1,Aurora Capital,0.000000,0.341033,0
2,BMO Capital,0.000000,0.077906,0
3,BTIG,1.000000,0.064953,1
4,Bank of the West,0.000000,0.145039,0
5,Barclays Capital,1.000000,0.038241,1
6,Benjamin Jerold,0.000000,0.336358,0
7,Boenning Scattergood,1.000000,0.102326,1
8,Bull Market Securities,0.000000,0.306412,0
9,Bulltick,0.000000,0.159728,0


In [23]:
data_panel.query('Rebate_Dummy == 1 & OrderType == "Market"')['Broker'].unique()

array(['Deutsche', 'Credit Suisse', 'Barclays Capital', 'BTIG',
       'TD Ameritrade', 'Boenning Scattergood', 'INTL FCStone',
       'JP Morgan', 'Evercore Group'], dtype=object)

In [24]:
data_606_mktshr.loc(:,'MktShare')

SyntaxError: invalid syntax (<ipython-input-24-3a90448a1ba1>, line 1)

In [25]:
temp = data_606.query('Broker == "Investment Placement Group"').copy()

brk = 'Investment Placement Group'

for mkt in temp['MarketCenter'].unique():
    
    temp2 = temp.query('MarketCenter == "' + mkt + '"')
    print(mkt)
    print(temp2['MktShare'].sum() / (3*3*len(data_606[data_606['Broker'] == brk]['Quarter'].unique())*len(data_606[data_606['Broker'] == brk]['MarketCenter'].unique())))

In [26]:
data_606.query('MarketCenter == "AQUA" & MktShare != 0')

,Broker,Exchange,OrderType,Quarter,Rebate,MarketCenter,MktShare,Rebate_Dummy,Obs_id
8841,Deutsche,Other,Other,2016Q3,1.0,AQUA,0.0001,1,Deutsche-AQUA-Other-Other
8860,Deutsche,Other,Other,2017Q1,1.0,AQUA,0.0001,1,Deutsche-AQUA-Other-Other
8935,Deutsche,Other,Total,2016Q3,1.0,AQUA,0.0001,1,Deutsche-AQUA-Other-Total
8954,Deutsche,Other,Total,2017Q1,1.0,AQUA,0.0001,1,Deutsche-AQUA-Other-Total
